In [9]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig, get_peft_model

BASE_MODEL_PATH = '/workspace/KoAlpaca_5b/model'
PEFT_PATH = '/workspace/KoAlpaca_5b/model_tuned'
TOKENIZER_PATH = '/workspace/KoAlpaca_5b/tokenizer'

peft_config = PeftConfig.from_pretrained(PEFT_PATH)
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)
model_base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
model_base.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 15.46it/s]


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 4096)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-27): 28 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)

In [10]:
model_to_merge = PeftModel.from_pretrained(model_base,PEFT_PATH)
merged_model = model_to_merge.merge_and_unload()
merged_model.eval()

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 4096)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-27): 28 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)

In [11]:
pipe_base = pipeline(
    'text-generation', 
    model=model_base,
    tokenizer=tokenizer,
    device = 0
)
pipe_merged = pipeline(
    'text-generation', 
    model=merged_model,
    tokenizer=tokenizer,
    device = 0
)

def ask_base(x, context='', is_input_full=False,):
        
    ans = pipe_base(
        f"### 질문: {x}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {x}\n\n### 답변:", 
        do_sample=True, 
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
        bos_token_id=0,
    )
    return ans[0]['generated_text']

def ask_merged(x, context='', is_input_full=False,):
            
        ans = pipe_merged(
            f"### 질문: {x}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {x}\n\n### 답변:", 
            do_sample=True, 
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.9,
            return_full_text=False,
            eos_token_id=2,
            bos_token_id=0,
        )
        return ans[0]['generated_text']


In [19]:
instrunction = "경국대전에 대해서 설명하세요."
context = ""
print(ask_base(instrunction, context))

print("--------")

print(ask_merged(instrunction, context))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 조선 시대에 관민을 대상으로 한 지배자의 공문서. 1474년(성종 15) 왕명으로 유효통 · 노중례 · 박윤덕 등이 간행하였으며, 김사형 · 이회 등에게 감수케 하여 1476년(성종 14)에 반포되었다. 활자로 인쇄하였으므로 조판인쇄방식에 따라 목판본으로 인출, 보관되었다. 고려시대에는 국자감 · 주자감 · 우정승상락감 · 우승지선공감 · 좌정승평양백에 임명되었으나, 조선시대에는 종부시 · 사옹원 · 내의원 · 상의원 · 사복시 ·군기시 ·사섬시 ·군자감 · 사옹원 · 사재감 · 군기감 · 의영고 · 장흥고 · 양화도 · 강화부 · 개경유수관원 · 하나로통신 · 한일합방에 공을 세우거나 의금부 · 전의감 · 비전의관 · 상의원 · 사복시 · 군기시 · 사재감 · 군기감 · 의영고 · 장흥고 · 양화도 · 강화부 · 개경유수관원 · 하나로통신 · 한일합방에 공을 세우거나 의금부 · 전의감 · 비전의관 · 상의원 · 사복시 · 군기시 · 사재감 · 군기감 · 의영고 · 장흥고 · 양화도 · 강화부 · 개경유수관원 · 하나로통신 · 한일합방에 공을 세우거나 의금부 · 전의감 · 비전의관 · 상의원 · 사복시 · 군기시 · 사재감 · 군기감 · 의영고 · 장흥고 · 양화도 · 강화부 · 개경유수관원 · 하나로통신 · 한일합방에 공을 세우거나 의금부 · 전의감 · 비전의관 · 상의원 · 사복시 · 군기시 · 사재감 · 군기감 · 의영고 · 장흥고 · 양화도 · 강화부 · 개경유수관원 · 하나로통신 · 한일합방에 공을 세우거나 의금부 · 전의감 · 비전의관 · 상의원 · 사복시 · 군기시 · 사재감 · 군기감 · 의영고 · 장흥고 · 양화도 · 강화부 · 개경유수관원 · 하나로통신
--------
 조선 왕조의 법전으로, 조선시대의 관법을 집대성한 책. 1644년(인조 22)에 완성하여 1667년(명종 8)에 간행하였다. 의의는 조선시대의 기본 법전이자, 한국 법전의 기초를 닦았다는 데에 있다. 내용은 계수관법 ·평관법 ·수세관법으로 구분하였고, 각 품계에 따라 율령 

In [20]:
import requests
import pandas as pd
def rag_input(question,n_results=10):
    url = 'http://143.248.90.7:8000/query-kv-top-n'
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
    }

    data = {
        'query': question,
        'n_results': n_results,
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        result = pd.DataFrame(response.json()) 
        result = result.sort_values(by=['similarity'], ascending=False)
        context = ' '.join(result['document'].tolist())
        context = context.replace('nan','')
        context = context.replace('\n','')
        context = context.replace("?-?",'')
        print(context)
    else:
        print(f"Error: {response.status_code}, {response.text}")
        context = None

    
    return context


In [21]:
context = rag_input(instrunction,n_results=2)
print(ask_base(instrunction, context))
print("-----")
print(ask_merged(instrunction, context))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


                           id  \
0  k_hist_thesaurus.csv_58054   
1  k_hist_thesaurus.csv_58055   

                                            document  similarity  
0  경국대전 2 經國大典 nan nan 1460 조선전기 서명 조선시대 통치 및 행정의...    0.854825  
1  경국대전주해 2 經國大典註解   nan 1555 조선전기 서명 1555년(명종 10...    0.788208  
경국대전 2 經國大典   1460 조선전기 서명 조선시대 통치 및 행정의 근간을 이룬 법전. 경국대전주해 2 經國大典註解    1555 조선전기 서명 1555년(명종 10)에 ≪경국대전(經國大典)≫의 규정 중 해석하기 어려운 조문이나 용어를 주석한 책.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 경국대전은 1460년(성종 13)에 왕이 제정한 대한민국의 법전이다. 경국대전은 1484년(성종 13)에 왕이 제정한 조선의 법전인 ≪경국대전≫과 달리, 법전의 내용을 구문별로 분석하여 편찬한 것이다.

경국대전은 태조 4년(1395)에 처음 책이 간행되었으며, 1484년에 완성되었다. 이후 1488년(성종 13)에 ≪훈민정음≫과 함께 보다 완성된 모습으로 재탄생하였다. 이후 1495년(연산군 1)에는 이 완성되어 반포되었다. ≪경국대전≫은 이후 여러 차례의 수정과 편집을 거쳐 1498년(연산군 4)에 완성되었다. ≪경국대전≫은 폭군으로 불리는 연산군이 통치하던 시기에 중단되었다가,1506년(연산군 10)에 중종반정으로 인해 다시 만들어졌다. 이때에는 경국대전을 보다 완성된 모습으로 만들기 위해 기억법과 필법을 수정하였다. 이후 1510년(중종 6)에 왕이 수정한 ≪경국대전≫이 처음으로 간행되었다. 
-----
 조선시대에는 법전이 있었습니다. 경국대전이 그것인데, 이 법전은 1460년에 편찬된 『경국대전』의 해설서입니다. 경국대전은 1474년에 편찬된 『속대전』과 함께 조선시대의 법전을 대표하는 책입니다. 경국대전은 대전통편, 통편, 평전, 지전 등으로 구성되어 있으며, 법전 중에서도 해석하기 어려운 부분들을 주석하고 있습니다. 조선시대에는 의부법전, 내령법전 등도 편찬되었습니다. 


In [16]:
context

'을사늑약 2 乙巳勒約   1905 대한제국기 외교·국제관계>일본>외교·국제관계(일본)>대한제국기 1905년 일본이 한국의 외교권을 박탈하기 위해 이완용을 비롯한 을사5적과 체결한 조약이나 고종 황제가 끝까지 재가하지 않은 원인 무효의 조약임. 을사늑약 2 乙巳勒約   1905 대한제국기 외교·국제관계>외교협상·협약>대한제국기 1905년 일본이 한국의 외교권을 박탈하기 위해 이완용을 비롯한 을사5적과 체결한 조약이나 고종 황제가 끝까지 재가하지 않은 원인 무효의 조약임.'